In [1]:
from fractions import Fraction
from typing import Tuple, Union
import numpy as np
import pandas as pd

pd.set_option('display.precision', 12)  # Increase decimal precision
pd.set_option('display.width', 300)     # Wider display
pd.set_option('display.max_columns', None)  # Show all column

In [2]:
def input_matrix(filename, convert_fractions=False):
    """
    Reads a matrix from a text file and returns it as a NumPy array.
    Supports fractional entries if present (though your files use decimals).
    """
    matrix = []

    with open(filename, 'r') as f:
        for line in f:
            tokens = line.strip().split()
            if not tokens:
                continue

            row = []
            for token in tokens:
                if '/' in token:
                    # convert fractions if any includes fraction sign
                    val = Fraction(token)
                    row.append(float(val) if convert_fractions else val)
                else:
                    # parse as float directly
                    row.append(float(token))

            matrix.append(row)
            
    dtype = float if convert_fractions else object
    return np.array(matrix, dtype=dtype)


In [3]:
def output_matrix(X: np.ndarray, precision: int = 12):
    """
    Prints a NumPy array (vector or matrix) in a clean tabular format using pandas.
    
    Parameters:
    - X: np.ndarray, 1D or 2D array.
    - precision: number of decimal places to round floats to.
    """
    # Wrap 1D arrays into a 2D DataFrame for consistent display
    if X.ndim == 1:
        df = pd.DataFrame(X, columns=["value"])
    elif X.ndim == 2:
        df = pd.DataFrame(X)
    else:
        raise ValueError("Only 1D or 2D arrays are supported.")
    
    # Round floats
    df = df.round(precision)
    # Print without index/header for cleaner look
    print(df.to_string(index=False, header=False))

Phương pháp Viền quanh tìm ma trận nghịch đảo

## Thuật toán

In [4]:
def compute_theta(A: np.ndarray, B_prev: np.ndarray, k: int) -> float:
    row = A[k-1, :k-1]
    col = A[:k-1, k-1]
    a_kk = A[k-1, k-1]
    theta = float(row @ B_prev @ col - a_kk)
    if np.isclose(theta, 0):
        raise ZeroDivisionError(f"θ_{k} is zero; block singular at size {k}.")
    return theta

In [5]:
def compute_b_nn(A: np.ndarray, B_prev: np.ndarray, k: int) -> float:
    # b_{k,k} = -1/θ_k
    theta = compute_theta(A, B_prev, k)
    return -1.0 / theta

In [6]:
def compute_beta_col(A: np.ndarray, B_prev: np.ndarray, k: int) -> np.ndarray:
    # β_{1,k-1}: last column block (excluding b_{k,k})
    alpha_col = A[:k-1, k-1]
    theta = compute_theta(A, B_prev, k)
    return B_prev.dot(alpha_col) / theta

In [7]:
def compute_beta_row(A: np.ndarray, B_prev: np.ndarray, k: int) -> np.ndarray:
    # β_{k-1,1}: last row block (excluding b_{k,k})
    alpha_row = A[k-1, :k-1]
    theta = compute_theta(A, B_prev, k)
    return alpha_row.dot(B_prev) / theta

In [8]:
def update_top_left_block(A: np.ndarray, B_prev: np.ndarray, k: int, beta_row: np.ndarray) -> np.ndarray:
    # B_new = B_prev @ (I - α_col ⊗ β_row)
    alpha_col = A[:k-1, k-1]
    I = np.eye(k-1)
    return B_prev.dot(I - np.outer(alpha_col, beta_row))

In [9]:
def block_matrix_recursion(A: np.ndarray) -> np.ndarray:
    """
    Computes A^{-1} via block recursive inversion on leading principal minors.
    """
    n = A.shape[0]
    # Base case k=1
    theta1 = A[0,0]
    if np.isclose(theta1, 0):
        raise ZeroDivisionError("θ_1 is zero; A[0,0] is singular.")
    B_prev = np.array([[1.0/theta1]])
    # Recursively build inverse for k=2..n
    for k in range(2, n+1):
        print(f"Theta {k}:", compute_theta(A, B_prev, k))
        # Compute new blocks
        beta_col = compute_beta_col(A, B_prev, k)
        beta_row = compute_beta_row(A, B_prev, k)
        b_kk = compute_b_nn(A, B_prev, k)
        # Update top-left (k-1)x(k-1)
        B_tl = update_top_left_block(A, B_prev, k, beta_row)
        # Assemble B_k
        Bk = np.zeros((k, k))
        Bk[:k-1, :k-1] = B_tl
        Bk[:k-1, k-1] = beta_col
        Bk[k-1, :k-1] = beta_row
        Bk[k-1, k-1] = b_kk
        B_prev = Bk
        print(f"Size {k}:\n", B_prev, "\n")
    return B_prev

In [10]:
def inverse_via_ata(A: np.ndarray) -> np.ndarray:
    """
    Computes A^{-1} indirectly by inverting M = A^T A via block recursion,
    then A^{-1} = M^{-1} A^T.
    """
    M = A.T.dot(A)
    M_inv = block_matrix_recursion(M)
    return M_inv.dot(A.T)

## Kết quả

In [11]:
#Original matrix Ax=B
A = input_matrix('BLMT_input_A.txt', convert_fractions=False)

print("\nMatrix A:"); output_matrix(A)
print("\nMatrix M = A^T * A:"); output_matrix(A.T.dot(A))


Matrix A:
 0.0  0.0  0.0  1.0
 1.0  2.0  3.0  4.0
-2.0  3.0  5.0 -1.0
 3.0  1.0 -4.0  8.0

Matrix M = A^T * A:
 14.0  -1.0 -19.0  30.0
 -1.0  14.0  17.0  13.0
-19.0  17.0  50.0 -25.0
 30.0  13.0 -25.0  82.0


In [12]:
A_inv = inverse_via_ata(A)
print("Inverse of A result:\n", A_inv)

Theta 2: -13.928571428571429
Size 2:
 [[0.07179487 0.00512821]
 [0.00512821 0.07179487]] 

Theta 3: -6.646153846153851
Size 3:
 [[ 0.31712963 -0.21064815  0.19212963]
 [-0.21064815  0.26157407 -0.16898148]
 [ 0.19212963 -0.16898148  0.15046296]] 

Theta 4: -1.0000000000000284
Size 4:
 [[ 4.20679012  2.36419753 -0.19135802 -1.97222222]
 [ 2.36419753  1.96604938 -0.42283951 -1.30555556]
 [-0.19135802 -0.42283951  0.1882716   0.19444444]
 [-1.97222222 -1.30555556  0.19444444  1.        ]] 

Inverse of A result:
 [[-1.9722222222221664 0.4722222222222321 -0.3055555555555447
  -0.027777777777778567]
 [-1.305555555555517 -0.1944444444444411 0.3611111111111158
  0.3055555555555518]
 [0.19444444444443823 0.3055555555555548 -0.1388888888888893
  -0.19444444444444442]
 [0.9999999999999716 -1.3322676295501878e-15 -5.440092820663267e-15
  4.440892098500626e-15]]
